# **Projet 4 : Anticipez les besoins en consommation électrique de bâtiments**

- Data : on a deux bases de données concernant les bâtiments de Seattle pour les années de 2015 et 2016.
- OBJET :
  - prédire les **émissions de CO2** et la **consommation totale d’énergie** de bâtiments pour lesquels elles n’ont pas encore été mesurées.
  - Evaluer l'interet de Energy star score


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sp
import sklearn

chargement des données

In [ ]:
downloaded = drive.CreateFile({'id':'1W2dC9esD3VEq4uackezRVviuu_wG1QOG'})
downloaded.GetContentFile('2016-building-energy-benchmarking.csv')
df16 = pd.read_csv('2016-building-energy-benchmarking.csv', sep = ',', header = None)

In [ ]:
df16.shape

In [ ]:
df16= df16.rename(columns=df16.iloc[0])
df16= df16.drop([0])

In [ ]:
df16.head(3)

In [ ]:
df16.dtypes.value_counts()

In [ ]:
df16.isna().sum()

In [ ]:
df16.nunique()

In [ ]:
downloaded = drive.CreateFile({'id':'1Ejb-snEwVaCINsTcrRw_qJl0zd55X5Yg'})
downloaded.GetContentFile('2015-building-energy-benchmarking.csv')
df15 = pd.read_csv('2015-building-energy-benchmarking.csv', sep = ',', header = None)

In [ ]:
df15.shape

In [ ]:
df15= df15.rename(columns=df15.iloc[0])
df15= df15.drop([0])

In [ ]:
df15.dtypes.value_counts()

In [ ]:
df15.isna().sum()

In [ ]:
df15.nunique()

In [ ]:
set(df16.columns).intersection(set(df15.columns))

Columns in df16 not in df15

In [ ]:
df16.columns.difference(df15.columns)

Columns in df15 not in df16

In [ ]:
df15.columns.difference(df16.columns)

meme colonnes dans df15 et df16 mais differents nom comme:
- GHGEmissionsIntensity(kgCO2e/ft2) et GHGEmissionsIntensity
- Comment et Comments
- Zip Codes et ZipCode
-TotalGHGEmissions et GHGEmissions(MetricTonsCO2e)


In [ ]:
#sans mentionné la grandeur
df15.rename(columns={'GHGEmissionsIntensity(kgCO2e/ft2)': 'GHGEmissionsIntensity'}, inplace=True)

**Vérification des id**

on a des bâtiments communs aux 2 années (2015 et 2016)

In [ ]:
# merge on id
df = df15.merge(df16, on='OSEBuildingID', how='left').ffill(axis=1)

In [ ]:
df['GHGEmissionsIntensity_y'].replace('Non-Compliant','0',inplace=True)
df['GHGEmissionsIntensity_y'].replace('Compliant','0',inplace=True)

In [ ]:
 df['GHGEmissionsIntensity_y']= df['GHGEmissionsIntensity_y'].astype(float)
df['GHGEmissionsIntensity_x'] = df['GHGEmissionsIntensity_x'].astype(float)
df['SiteEnergyUseWN(kBtu)_x'] = df['SiteEnergyUseWN(kBtu)_x'].astype(float)
df['SiteEnergyUseWN(kBtu)_y'] = df['SiteEnergyUseWN(kBtu)_y'].astype(float)

In [ ]:
df16.set_index('OSEBuildingID',inplace=True)
df.set_index('OSEBuildingID',inplace=True)
df15.set_index('OSEBuildingID',inplace=True)

Pour les batiments communs : Supprimer les batiments dont les valeurs des variables cibles sont proches

In [ ]:

# filter
df15_new= df15[((df['SiteEnergyUseWN(kBtu)_x'] / df['SiteEnergyUseWN(kBtu)_y'])<= 0.3) |((df['SiteEnergyUseWN(kBtu)_x'] / df['SiteEnergyUseWN(kBtu)_y'])>= 1.2) |((df['GHGEmissionsIntensity_x'] / df['GHGEmissionsIntensity_y'])<= 0.3) |((df['GHGEmissionsIntensity_x'] / df['GHGEmissionsIntensity_y'])>= 1.2) ]

merge de deux dataframe 2015 et 2016 : uniformisation

In [ ]:
df = pd.concat([df15_new, df16], axis=0, sort=False)

In [ ]:
df.head(10)

- les variables cibles :
 - variable energie : SiteEnergyUseWN(kBtu)  
 - Variable emissions : GHGEmissionsIntensity

**Etude** **des** **valeurs** **manquantes**

In [ ]:
#calcul le taux de NaN pour chaque variable
df_nan = df.isna().sum().sort_values(ascending=False)
plt.figure(figsize=(10,15))
plt.title('Proportion de NaN par variable (%)')
sns.barplot(y=df_nan.index, x=df_nan.values/df.shape[0]*100)

Supprimer les colonnes dont le nombre de NaN est superieur à 85%

In [ ]:
df = df.loc[:, df.isnull().mean() <= .85]

In [ ]:
df.isna().sum().sort_values()

supprimer les colonnes inutiles pour la prediction

In [ ]:
columns_drop =['City', 'State', 'PropertyName', 'DefaultData', 'Address' ,'ZipCode','ComplianceStatus','DataYear',
 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)', 'SteamUse(kBtu)', 'Electricity(kWh)','Latitude','Longitude',
'Electricity(kBtu)', 'NaturalGas(therms)', 'NaturalGas(kBtu)','TotalGHGEmissions','TaxParcelIdentificationNumber','Neighborhood','CouncilDistrictCode' ]
df=df.drop(columns_drop,axis=1)

In [ ]:
df['GHGEmissionsIntensity']= df['GHGEmissionsIntensity'].replace(['0'],np.nan)

In [ ]:
df['SiteEnergyUseWN(kBtu)']=df['SiteEnergyUseWN(kBtu)'].replace(['0'],np.nan)

supprimer les batiments sans GHGEmissionsIntensity renseignes

In [ ]:
df.dropna(subset=['GHGEmissionsIntensity'],inplace=True)
df['GHGEmissionsIntensity'].isna().sum()

supprimer les batiments sans SiteEnergyUseWN(KBtu) renseignes

In [ ]:
df.dropna(subset=['SiteEnergyUseWN(kBtu)'],inplace=True)
df['SiteEnergyUseWN(kBtu)'].isna().sum()

Distribution des variables cibles

In [ ]:
cible_col = df[['SiteEnergyUseWN(kBtu)','GHGEmissionsIntensity']]
for column in cible_col:
    plt.figure(figsize = (10,10))
    sns.distplot(df[column],axlabel=column)
    plt.title('Distrubtion de la variable cible : {}'.format(column))
    plt.show()

In [ ]:
from scipy.stats import skew
df['SiteEnergyUseWN(kBtu)'] = df['SiteEnergyUseWN(kBtu)'].astype(float)
df['GHGEmissionsIntensity'] = df['GHGEmissionsIntensity'].astype(float)
print(skew(df['SiteEnergyUseWN(kBtu)']))
print(skew(df['GHGEmissionsIntensity']))

 - Valeur de skewenss superieur à 1 alors on peut passer au log

 Application du transformation logarithmique du variable cible afin d'obtenir une distribution qui se rapproche d'une loi normale

In [ ]:
df['log_SiteEnergyUseWN(kBtu)'] = np.log(df['SiteEnergyUseWN(kBtu)'])
df['log_GHGEmissionsIntensity'] = np.log(df['GHGEmissionsIntensity'])


In [ ]:
df['log_SiteEnergyUseWN(kBtu)'].isna().sum()

In [ ]:
df['log_GHGEmissionsIntensity'].isna().sum()

distribution de nouvelle varible cible ( avec log)

In [ ]:
df['log_GHGEmissionsIntensity'] = df['log_GHGEmissionsIntensity'].mask(np.isinf(df['log_GHGEmissionsIntensity']))

In [ ]:
df.dropna(subset=['log_GHGEmissionsIntensity'],inplace=True)

In [ ]:
df['log_GHGEmissionsIntensity'].isna().sum()

In [ ]:
new_cible_col = df[['log_SiteEnergyUseWN(kBtu)','log_GHGEmissionsIntensity']]
for column in new_cible_col:
    plt.figure(figsize = (10,10))
    sns.distplot(df[column],axlabel=column)
    plt.title('Distrubtion de la variable cible : {}'.format(column))
    plt.show()

**Etude des variables catégorielles et numerique**

- columns to categorize :
 -  BuildingType
 -  PrimaryPropertyType
 - ListOfAllPropertyUseTypes
 - LargestPropertyUseType
 - SecondLargestPropertyUseType
 - ThirdLargestPropertyUseType  
---



Pour transformer ces variables catégorielles en variables numeriques , on va utiliser deux methodes : **One Hot Encoding (OHE)** et **Target encoding**

 - One Hot Encoding (OHE) consiste à créer autant de colonnes qu'il y a de catégories (mettre des 0 partout sauf dans la colonne correspondant à la catégorie du bâtiment où on met 1)
 - Target encoding consiste à  remplacer la catégorie par la valeur moyenne de la variable cible des bâtiments de la même catégorie

In [ ]:
df['PrimaryPropertyType'].unique()

Valeurs quasi identiques

In [ ]:
df['PrimaryPropertyType'].replace('Restaurant\n', 'Restaurant',inplace=True)
df['PrimaryPropertyType'].replace('Distribution Center\n','Distribution Center',inplace=True)
df['PrimaryPropertyType'].replace('Supermarket / Grocery Store','Supermarket/Grocery Store',inplace=True)
df['PrimaryPropertyType'].replace('Residence Hall/Dormitory','Residence Hall',inplace=True)
df['PrimaryPropertyType'].replace('Self-Storage Facility\n','Self-Storage Facility',inplace=True)

In [ ]:
df['BuildingType'].unique()

In [ ]:
df['ListOfAllPropertyUseTypes'].unique()

In [ ]:
df['LargestPropertyUseType'].unique()

In [ ]:
df['SecondLargestPropertyUseType'].unique()

In [ ]:
df['ThirdLargestPropertyUseType'].unique()

In [ ]:
#Le nombre de bâtiments ne peut être nul
df["NumberofBuildings"].unique()

Imputation de valeurs manquantes pour les colonnes numeriques par la moyenne de Property type

In [ ]:
num_columns=['ThirdLargestPropertyUseTypeGFA','LargestPropertyUseTypeGFA','SecondLargestPropertyUseTypeGFA']
df[num_columns] = df[num_columns].astype('float')

In [ ]:
df['LargestPropertyUseTypeGFA'] = df['LargestPropertyUseTypeGFA'].fillna(df.groupby('PrimaryPropertyType')['LargestPropertyUseTypeGFA'].transform('mean'))

In [ ]:
df['SecondLargestPropertyUseTypeGFA'] = df['SecondLargestPropertyUseTypeGFA'].fillna(df.groupby('PrimaryPropertyType')['SecondLargestPropertyUseTypeGFA'].transform('mean'))

In [ ]:
df['ThirdLargestPropertyUseTypeGFA'] = df['ThirdLargestPropertyUseTypeGFA'].fillna(df.groupby('PrimaryPropertyType')['ThirdLargestPropertyUseTypeGFA'].transform('mean'))

In [ ]:
df['LargestPropertyUseTypeGFA'].isna().sum()

In [ ]:
df['SecondLargestPropertyUseTypeGFA'].isna().sum()

In [ ]:
df['ThirdLargestPropertyUseTypeGFA'].isna().sum()

In [ ]:
df.dropna(subset=['NumberofFloors'],inplace=True)
df['NumberofFloors'].isna().sum()

In [ ]:
df.dropna(subset=['ThirdLargestPropertyUseTypeGFA'],inplace=True)
df.dropna(subset=['SecondLargestPropertyUseTypeGFA'],inplace=True)

In [ ]:
df.head(5)

Proportion des varaibles categoriels

In [ ]:
import matplotlib.pyplot as plt
ct_col = df[['BuildingType','PrimaryPropertyType','ListOfAllPropertyUseTypes','LargestPropertyUseType','SecondLargestPropertyUseType','ThirdLargestPropertyUseType']]
for column in ct_col:
    plt.figure(figsize = (7,7))
    df[column].value_counts(normalize=True).plot(kind='pie')
    #df[column].value_counts(normalize=True).plot(kind='bar')
    plt.title('proportion du variable : {}'.format(column))

    plt.show()



In [ ]:
for column in ct_col:
    plt.figure(figsize = (7,7))
    df[column].value_counts(normalize=True).plot(kind='bar')
    plt.title('proportion du variable : {}'.format(column))

    plt.show()

In [ ]:
columns_to_categorize = ['BuildingType','PrimaryPropertyType','ListOfAllPropertyUseTypes','LargestPropertyUseType','SecondLargestPropertyUseType','ThirdLargestPropertyUseType']
df[columns_to_categorize] = df[columns_to_categorize].astype('category')
df.dtypes


 **One Hot Encoder**

In [ ]:
col_ohe = ['BuildingType','PrimaryPropertyType']
from sklearn.preprocessing import OneHotEncoder, StandardScaler
ohe = OneHotEncoder(sparse=False)
transformed = ohe.fit_transform(df[col_ohe])
data= pd.DataFrame(transformed,columns=ohe.get_feature_names().tolist())
df.reset_index(inplace=True)
df= df.drop(['BuildingType','PrimaryPropertyType'],axis=1)
#concat with the original data
df= pd.concat([df,data],axis=1)

In [ ]:
df.head(5)

**Target Encoding**

In [ ]:
from sklearn.model_selection import train_test_split
cols_te= ['ListOfAllPropertyUseTypes','LargestPropertyUseType','SecondLargestPropertyUseType','ThirdLargestPropertyUseType']
df1=df.copy()
X=df1
Y = df1[['log_SiteEnergyUseWN(kBtu)']]
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, train_size=.8,random_state=10)

In [ ]:
pip install --upgrade category_encoders

In [ ]:
#Target Encoding: encoder la variable catégorielle avec la moyenne de la cible
import category_encoders as ce
encoder1 = ce.TargetEncoder(cols=cols_te)
X_target_encoded1= encoder1.fit_transform(xtrain,ytrain) #(df.Categ,df.Target)
X_target_encoded1.sample(10)

In [ ]:
df2=df.copy()
#X= df1.drop(columns=['log_SiteEnergyUseWN(kBtu)','log_GHGEmissionsIntensity'])
X2=df2
Y2 = df2[['log_GHGEmissionsIntensity']]
x_train, x_test, y_train, y_test = train_test_split(X2, Y2, train_size=.8,random_state=10)

In [ ]:
encoder2 = ce.TargetEncoder(cols=cols_te)
X_target_encoded2= encoder2.fit_transform(x_train,y_train)
X_target_encoded2.sample(10)

In [ ]:
X_target_encoded2.drop(['SiteEnergyUseWN(kBtu)','GHGEmissionsIntensity'],axis=1,inplace=True)

In [ ]:
X_target_encoded1.drop(['SiteEnergyUseWN(kBtu)','GHGEmissionsIntensity'],axis=1,inplace=True)

df1 pour predire "log_SiteEnergyUseWN(kBtu)"

In [ ]:
df1= X_target_encoded1.copy()

In [ ]:
df1= df1.drop(['log_GHGEmissionsIntensity'],axis=1)

df2 pour predire "log_GHGEmissionsIntensity"

In [ ]:
df2= X_target_encoded2.copy()

In [ ]:
df2= df2.drop(['log_SiteEnergyUseWN(kBtu)'],axis=1)

Transformation des types numeriques  

In [ ]:
num_cols= ['OSEBuildingID','YearBuilt','NumberofBuildings','NumberofFloors','PropertyGFATotal','PropertyGFAParking','PropertyGFABuilding(s)']
df1[num_cols] = df1[num_cols].astype('float')
df1.dtypes

In [ ]:
num= ['OSEBuildingID','YearBuilt','NumberofBuildings','NumberofFloors','PropertyGFATotal','PropertyGFAParking','PropertyGFABuilding(s)']
df2[num] = df2[num].astype('float')
df2.dtypes

 **Modélisation**

les modelés à étudiés sont :
- Elastic Net regression
- Support vector regressor (SVR)
- Bagging : Random forest
- Boosting : Gradient Boosting (XGBoost)

PROCESSUS :

- Entrainons les modèles sans le variable d'Energy Star Score
-Comparaison des differentes modéles selon le score de GridSearch ( le score de cross validation )
- Selectionne du modele finale
- Entraine le modele finale uniquement avec les batiments qui ont un energystarscore renseignes.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

**1- Séparation du jeu de données en Train et Test**

In [ ]:
X= df1.drop(['log_SiteEnergyUseWN(kBtu)','ENERGYSTARScore'],axis=1)
Y= df1['log_SiteEnergyUseWN(kBtu)']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
# standardiser les données
std_scale = StandardScaler()
X_train = std_scale.fit_transform(X_train) #X_train_std
X_test = std_scale.transform(X_test)#X_test_std

**Regression Elastic Net**

- Le modèle Elastic Net comprend les modèles linéaires ridge et lasso via les valeurs des coefficients de penalite L1 ratio.

In [ ]:
from sklearn.linear_model import ElasticNet
# grille de paramètres:
#alpha exprimer la valeur de lambda qui mesure la complexite du model
#L1 ratio est la penalite = 1 équivaut à un Lasso, 0 à un Ridge
param_grid = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10], 'l1_ratio':np.arange(0.0, 1.0, 0.1) }
# critère de sélection du meilleur modèle :RMSE
score = 'neg_mean_squared_error'
# initialiser une recherche sur grille
grid_elastic1 = GridSearchCV(estimator = ElasticNet(),param_grid= param_grid,scoring= score,cv=5,verbose=0)
# Entrainement
grid_elastic1.fit(X_train, Y_train)
# afficher les paramètres optimaux
print("The optimal parameters are {} with a score of {:.2f}".format(grid_elastic1.best_params_, grid_elastic1.best_score_))

Le terme l1_ratio est 0.1, signifiant que le poids du L2 (ridge) est plus important que celui du L1 (Lasso) dans la régularisation

In [ ]:
# grille de paramètres:
#alpha exprimer la valeur de lambda qui mesure la complexite du model
#L1 ratio = 1 équivaut à un Lasso, 0 à un Ridge
param_grid = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10],'l1_ratio':np.arange(0.0, 1.0, 0.1) }
# critère de sélection du meilleur modèle :R2
score = 'r2'
# initialiser une recherche sur grille
grid_elastic2 = GridSearchCV(estimator = ElasticNet(),param_grid= param_grid,scoring= score,cv=5,verbose=0)
# Entrainement
grid_elastic2.fit(X_train, Y_train)
# afficher les paramètres optimaux
print("The optimal parameters are {} with a score of {:.2f}".format(grid_elastic2.best_params_, grid_elastic2.best_score_))

In [ ]:
# grille de paramètres:
#alpha exprimer la valeur de lambda qui mesure la complexite du model
#L1 ratio = 1 équivaut à un Lasso, 0 à un Ridge
param_grid = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10], 'l1_ratio':np.arange(0.0, 1.0, 0.1) }
# critère de sélection du meilleur modèle :R2
score = 'neg_mean_absolute_error'
# initialiser une recherche sur grille
grid_elastic3 = GridSearchCV(estimator = ElasticNet(),param_grid=param_grid, scoring= score,cv=5,verbose=0)
# Entrainement
grid_elastic3.fit(X_train, Y_train)
# afficher les paramètres optimaux
print("The optimal parameters are {} with a score of {:.2f}".format(grid_elastic3.best_params_, grid_elastic3.best_score_))

In [ ]:
import math
result= pd.DataFrame({
    'Modèle' : ['Elasticnet Regression'],
    'Score_RMSE_test':[math.sqrt(mean_squared_error(grid_elastic1.predict(X_test), Y_test))],
    'Score_R2_test':r2_score(grid_elastic2.predict(X_test), Y_test),
    'Score_MAE_test':mean_absolute_error(grid_elastic3.predict(X_test), Y_test)
})

In [ ]:
 result_score= pd.DataFrame({
    'Modèle' : ['Elasticnet Regression'],
    'Score_RMSE':grid_elastic1.best_score_,
    'Score_R2':grid_elastic2.best_score_,
    'Score_MAE':grid_elastic3.best_score_
})


In [ ]:
result

In [ ]:
result_score

**Random forest**

In [ ]:
#min_samples_split:est nombre minimal d'exemples que le modèle doit avoir pour pouvoir faire une nouvelle séparation.
#criterion : choisir entre deux methodes pour calculer l'erreur de notre modèle lors de son entraînement : entropy'ou 'gini' criterion='mae'

#min_sample_leaf : est le minimum d'exemples requis pour créer une feuille.(nombre de feuilles minimales dans un node)
#max_features : est le nombre de variables à tester pour chaque arbre :
      #-option auto qui effectue un test pour chaque variable du jeu de données
      #-option sqrt qui teste la racine carrée du nombre de variables du jeu de données.
      #-option log2 qui teste le log2 du nombre de variables du jeu de données.


#max_leaf_nodes : est le nombre maximum de feuilles de notre arbre (nombre maxiamle de decision puisque Les décisions sont prises dans les feuilles??? )
#n_estimators' : est le nombre d'arbres de décision
#max_depth: est la profondeur de l'arbre ( plus l'arbre est profond plus il sera complexe et plus il souffrira du sur-entraînement.)
#plus le modèle sera simple et généralisable. Cette meilleure généralisation augmentera les performances du modèle sur le jeu de test.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
param_grid= {'max_depth': [5,10,15,20],'max_features':['auto', 'sqrt','log2'],'min_samples_leaf':[1,3,5,10]}
score='neg_mean_squared_error'
grid_Rforest = GridSearchCV(estimator = RandomForestRegressor(n_estimators=500,random_state=123),param_grid= param_grid,scoring= score,cv=5,verbose=0)
grid_Rforest.fit(X_train, Y_train)

Le modèle optimal a les caractéristiques suivantes:

In [ ]:
grid_Rforest.best_params_

In [ ]:
grid_Rforest.best_estimator_

In [ ]:
# l'importance des variables du jeu de données dans la construction des arbres de la forêt.
df_rf= pd.DataFrame({'Features' : X.columns,'Features_importances' : grid_Rforest.best_estimator_.feature_importances_})

In [ ]:
df_rf.sort_values(by='Features_importances', ascending=False,inplace=True)

In [ ]:
plt.figure(figsize=(10,10))
plt.title('RandomForestRegressor - Importance des Features')
sns.barplot(y = df_rf['Features'],x = (df_rf['Features_importances']))
plt.show()

In [ ]:
param_grid= {'max_depth': [5,10,15] ,'max_features':['auto', 'sqrt','log2'] ,'min_samples_leaf':[1,3,5,10,15]}
score='r2'
grid_Rforest2 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=500,random_state=123),param_grid=param_grid,scoring= score,cv=5,verbose=0)
grid_Rforest2.fit(X_train, Y_train)

In [ ]:
grid_Rforest2.best_params_

In [ ]:
param_grid= {'max_depth': [5,10,15] ,'max_features':['auto', 'sqrt','log2'] ,'min_samples_leaf':[1,3,5,10,15]}
score='neg_mean_absolute_error'
grid_Rforest3 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=500),param_grid=param_grid,scoring= score,cv=5,verbose=0)
grid_Rforest3.fit(X_train, Y_train)

In [ ]:
grid_Rforest3.best_params_

In [ ]:
result = result.append(pd.DataFrame({
    'Modèle' : ['Random Forest Regressor'],
    'Score_RMSE_test' : [math.sqrt(mean_squared_error(grid_Rforest.predict(X_test),Y_test))],
    'Score_R2_test':r2_score(grid_Rforest2.predict(X_test),Y_test),
    'Score_MAE_test':mean_absolute_error(grid_Rforest3.predict(X_test),Y_test)
    }))


In [ ]:
result

In [ ]:
 result_score= result_score.append(pd.DataFrame({
    'Modèle' : ['Random Forest Regressor'],
    'Score_RMSE':grid_Rforest.best_score_,
    'Score_R2':grid_Rforest2.best_score_,
    'Score_MAE':grid_Rforest3.best_score_
}))


In [ ]:
result_score

En random forest on obtient une RMSE de 0,304, très légèrement inférieure à la valeur obtenue avec Elastic net regressor (0,471), de plus un score R2 plus eléve qu'on elastic net.

**SVM**

In [ ]:
 #séparateur (l’hyperplan de séparation) qui maximise la marge : séparateur à vaste marge
 #Kernel : noyau polynominal, quadratique, gauss (rbf : radial basis function)
 #svm(C=1,kernel='rbf',gamma=0.01,epsilon = )

If WE choose a SVM we wont be having feature importance parameter, but in decision trees we will get it

In [ ]:
from sklearn.svm import SVR #Support Vector Regression
param_grid= {'gamma' : [1e-2, 1e-3, 1e-4, 1e-5],
              'epsilon' : [0.001, 0.01, 0.1, 1], #erreur tolérée par l'algorithme
              'C' : [0.001, 0.10, 0.1, 10, 25, 50,100]} #terme de regularisation qui maximisez la marge et minimisez un terme d'erreur

svm_grid1= GridSearchCV(estimator = SVR(),param_grid=param_grid,scoring ='neg_mean_squared_error',cv=5,verbose=0)

svm_grid1.fit(X_train, Y_train)

In [ ]:
svm_grid1.best_params_

In [ ]:
param_grid= {'gamma' : [1e-1,1e-2, 1e-3, 1e-4, 1e-5], #kernel coefficient [ici kernel = Radial Basis Function]
              'epsilon' : [0.001, 0.01, 0.1, 1], #erreur tolérée par l'algorithme
              'C' : [0.001, 0.10, 0.1, 10, 25, 50]} #terme de regularisation qui maximisez la marge et minimisez un terme d'erreur

svm_grid2= GridSearchCV(estimator = SVR(),param_grid=param_grid,scoring ='r2',cv=5,verbose=0)

svm_grid2.fit(X_train,Y_train)

In [ ]:
svm_grid2.best_params_

In [ ]:
param_grid= {'gamma' : [1e-1,1e-2, 1e-3, 1e-4, 1e-5], #kernel coefficient [ici kernel = Radial Basis Function]
              'epsilon' : [0.001, 0.01, 0.1, 1], #erreur tolérée par l'algorithme
              'C' : [0.001, 0.10, 0.1, 10, 25, 50]} #terme de regularisation qui maximisez la marge et minimisez un terme d'erreur

svm_grid3= GridSearchCV(estimator = SVR(),param_grid=param_grid,scoring ='neg_mean_absolute_error',cv=5,verbose=0)

svm_grid3.fit(X_train,Y_train)

In [ ]:
svm_grid3.best_params_

In [ ]:
result = result.append(pd.DataFrame({
    'Modèle' : ['Support Vector Regressor'],
    'Score_RMSE_test' : [math.sqrt(mean_squared_error(svm_grid1.predict(X_test), Y_test))],
    'Score_R2_test':r2_score(svm_grid2.predict(X_test), Y_test),
    'Score_MAE_test':mean_absolute_error(svm_grid3.predict(X_test), Y_test)
    }))

In [ ]:
result

In [ ]:
 result_score= result_score.append(pd.DataFrame({
    'Modèle' : ['Support Vector Regressor'],
    'Score_RMSE':svm_grid1.best_score_,
    'Score_R2':svm_grid2.best_score_,
    'Score_MAE':svm_grid3.best_score_
}))

In [ ]:
result_score

**XG Boost**

In [ ]:
from xgboost import XGBRegressor
param_grid= {'n_estimators' : [10,50,100,500,1000,2000], 'learning_rate' : [1,0.1,0.01,0.001,0.0001]}
score = 'neg_mean_squared_error'
xgb_grid= GridSearchCV(XGBRegressor(n_jobs=-1),param_grid,scoring = score,cv = 5,verbose=0)
xgb_grid.fit(X_train,Y_train)

In [ ]:
xgb_grid.best_params_

In [ ]:
xgb_grid.best_estimator_

In [ ]:
xgb_grid.best_estimator_.feature_importances_

In [ ]:
df_xgb= pd.DataFrame({'Features' : X.columns,'Features_importances' : xgb_grid.best_estimator_.feature_importances_})

In [ ]:
df_xgb.sort_values(by='Features_importances', ascending=False,inplace=True)

In [ ]:
df_xgb

In [ ]:
#df_xgb.sort_values(by='Features_importances', ascending=False,inplace=True)
plt.figure(figsize=(10,10))
plt.title('XGB - Importance des Features')
sns.barplot(y = df_xgb['Features'],x = (df_xgb['Features_importances']))
plt.show()

In [ ]:
param_grid= {'n_estimators' : [100,500,1000,2000],'learning_rate' : [1,0.1,0.01,0.001,0.0001]}
score = 'r2'
xgb_grid2= GridSearchCV(XGBRegressor(n_jobs=-1),param_grid=param_grid,scoring = score,cv = 5,verbose=0)
xgb_grid2.fit(X_train,Y_train)

In [ ]:
xgb_grid2.best_params_

In [ ]:
xgb_grid2.best_estimator_

In [ ]:
from xgboost import XGBRegressor
param_grid= {'n_estimators' : [100,500,1000,2000],'learning_rate' : [1,0.1,0.01,0.001,0.0001]}
score = 'neg_mean_absolute_error'
xgb_grid3 = GridSearchCV(XGBRegressor(n_jobs=-1),param_grid= param_grid,scoring = score,cv = 5,verbose=0)
xgb_grid3.fit(X_train,Y_train)

In [ ]:
xgb_grid3.best_params_

In [ ]:
xgb_grid3.best_estimator_

In [ ]:

result = result.append(pd.DataFrame({
    'Modèle' : ['XGBoost'],
    'Score_RMSE_test' : [math.sqrt(mean_squared_error(xgb_grid.predict(X_test),Y_test))],
    'Score_R2_test':r2_score(xgb_grid2.predict(X_test), Y_test),
    'Score_MAE_test':mean_absolute_error(xgb_grid3.predict(X_test),Y_test)
    }))

In [ ]:
result

In [ ]:
 result_score= result_score.append(pd.DataFrame({
    'Modèle' : ['XGBoost'],
    'Score_RMSE':xgb_grid.best_score_,
    'Score_R2':xgb_grid2.best_score_,
    'Score_MAE':xgb_grid3.best_score_
}))

Comparaison des modeles selon le critere de score

In [ ]:
result_score

In [ ]:
plt.figure(figsize=(10,4))
plt.title('Comparaison des performances des modèles (jeu de test)')
chart = sns.barplot(x = result['Modèle'],y = result['Score_RMSE_test'])
#chart.set_xticklabels(labels = results['Modèle'], rotation=45,horizontalalignment='right',size=12,)

plt.show()

Le temps de prediction pour chaque modele :

In [ ]:
%timeit svm_grid3.predict(X_test)

In [ ]:
%timeit grid_Rforest3.predict(X_test)

In [ ]:
%timeit grid_elastic3.predict(X_test)

In [ ]:
%timeit xgb_grid3.predict(X_test)

l'algorithme qui predire  le plus rapidement est le XGBoost.

le temps de prediction de XGBoost est plus faible que celui de random forest ( 2 fois plus faible), ce pourrait être un critère pour choisir éventuellement le XGBoost plutôt que le Random Forest Regressor

In [ ]:
result.set_index(['Modèle'],inplace=True)

In [ ]:
result.T.plot(kind='bar',title='Comparaison des performances des modèles',figsize=(8,8))

In [ ]:
result_score.set_index(['Modèle'],inplace=True)

In [ ]:
result_score

pour sélectionner le model pour les étapes suivantes, on a fait une comparaison entre les scores et le temps de prediction

- GBoost et Random Forest Regressor sont les modèles avec la plus faible RMSE, avec une valeur très proche.
- l'algorithme XGBoost predit le plus rapidement que Random Forest

In [ ]:

result_score.T.plot(kind='bar',title='Comparaison des performances des modèles',figsize=(8,8))

In [ ]:
result_score.set_index(['Modèle'],inplace=True)

In [ ]:
result_score.set_index(['Modèle'],inplace=True)

In [ ]:
xgb_grid.best_params_

In [ ]:
xgb_grid.best_estimator_

- Entrainons du modèle selectionnée le XGBoost avec les mêmes données mais de plus le variable d'Energy Star Score ("ENERGYSTARScore")

In [ ]:
df1['ENERGYSTARScore'] = df1['ENERGYSTARScore'].astype('float')

In [ ]:
X1= df1.drop(['log_SiteEnergyUseWN(kBtu)'],axis=1)
Y1= df1['log_SiteEnergyUseWN(kBtu)']
xTrain, xTest, yTrain, yTest = train_test_split(X1, Y1, test_size=0.3)
# standardiser les données
std_scale = StandardScaler()
xTrain = std_scale.fit_transform(xTrain) #X_train_std
x_Test = std_scale.transform(xTest)#X_test_std

In [ ]:
param_grid= {'n_estimators' : [10,50,100,500,1000,2000,3000], 'learning_rate' : [1,0.1,0.01,0.001,0.0001]}
score = 'neg_mean_squared_error'
xgb_grid_ES= GridSearchCV(XGBRegressor(n_jobs=-1),param_grid,scoring = score,cv = 5,verbose=0)
xgb_grid_ES.fit(xTrain,yTrain)

In [ ]:
xgb_grid_ES.best_params_

In [ ]:
xgb_grid_ES.best_estimator_

In [ ]:
xgb_grid_ES.best_score_

La RMSE est plus petit que dans le cas du modèle sans energy score , le Energy Star Score améliore la performance et semble donc pertinent.

In [ ]:
df_xgb_ES= pd.DataFrame({'Features' : X1.columns,'Features_importances' : xgb_grid_ES.best_estimator_.feature_importances_})

In [ ]:
df_xgb_ES.sort_values(by='Features_importances', ascending=False,inplace=True)
plt.figure(figsize=(10,10))
plt.title('XGB - Importance des Features')
sns.barplot(y = df_xgb_ES['Features'],x = (df_xgb_ES['Features_importances']))
plt.show()

In [ ]:
%timeit xgb_grid_ES.predict(x_Test)

In [ ]:
math.sqrt(mean_squared_error(xgb_grid_ES.predict(x_Test),yTest))

In [ ]:
pred_y= xgb_grid_ES.best_estimator_.predict(x_Test)

In [ ]:
pd.DataFrame({'emissions' : yTest, 'prediction_emission' : pred_y})

In [ ]:
plt.figure(figsize=(8,8))
plt.title('Comparaison des emissions prédictes et réels')
sns.lineplot([10,20],[10,20], color='r')
sns.scatterplot(yTest,pred_y)
plt.show()

**Prediction de GHGEmissions Intensity**

1- sans Energyscore

In [ ]:
X2= df2.drop(['log_GHGEmissionsIntensity','ENERGYSTARScore'],axis=1)
Y2= df2['log_GHGEmissionsIntensity']
XTrain, XTest, YTrain, YTest = train_test_split(X2, Y2, test_size=0.3)
# standardiser les données
std_scale = StandardScaler()
XTrain = std_scale.fit_transform(XTrain) #X_train_std
XTest = std_scale.transform(XTest)#X_test_std

In [ ]:
from sklearn.linear_model import ElasticNet
param_grid = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10], 'l1_ratio':np.arange(0.0, 1.0, 0.1) }
# critère de sélection du meilleur modèle :RMSE
score = 'neg_mean_squared_error'

grid_elastic_1 = GridSearchCV(estimator = ElasticNet(),param_grid= param_grid,scoring= score,cv=5,verbose=0)

grid_elastic_1.fit(XTrain, YTrain)

print("The optimal parameters are {} with a score of {:.2f}".format(grid_elastic_1.best_params_, grid_elastic_1.best_score_))

In [ ]:
grid_elastic_1.best_estimator_

In [ ]:
param_grid = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10], 'l1_ratio':np.arange(0.0, 1.0, 0.1) }
score = 'r2'

grid_elastic_2 = GridSearchCV(estimator = ElasticNet(),param_grid= param_grid,scoring= score,cv=5,verbose=0)

grid_elastic_2.fit(XTrain, YTrain)

print("The optimal parameters are {} with a score of {:.2f}".format(grid_elastic_2.best_params_, grid_elastic_2.best_score_))

In [ ]:
param_grid = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10], 'l1_ratio':np.arange(0.0, 1.0, 0.1) }
score = 'neg_mean_absolute_error'

grid_elastic_3 = GridSearchCV(estimator = ElasticNet(),param_grid= param_grid,scoring= score,cv=5,verbose=0)

grid_elastic_3.fit(XTrain, YTrain)

print("The optimal parameters are {} with a score of {:.2f}".format(grid_elastic_3.best_params_, grid_elastic_3.best_score_))

In [ ]:
 result_sc= pd.DataFrame({
    'Modèle' : ['elastic net'],
    'Score_RMSE':grid_elastic_1.best_score_,
    'Score_R2':grid_elastic_2.best_score_,
    'Score_MAE':grid_elastic_3.best_score_

})

In [ ]:
from sklearn.ensemble import RandomForestRegressor
param_grid= {'max_depth': [5,10,15,20],'max_features':['auto', 'sqrt','log2'],'min_samples_leaf':[1,3,5,10]}
score='neg_mean_squared_error'
grid_Rforest_1 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=500,random_state=123),param_grid= param_grid,scoring= score,cv=5,verbose=0)
grid_Rforest_1.fit(XTrain, YTrain)

In [ ]:
grid_Rforest_1.best_params_

In [ ]:
grid_Rforest_1.best_estimator_

In [ ]:
grid_Rforest_1.best_score_

In [ ]:
param_grid= {'max_depth': [5,10,15,20],'max_features':['auto', 'sqrt','log2'],'min_samples_leaf':[1,3,5,10]}
score='r2'
grid_Rforest_2 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=500,random_state=123),param_grid= param_grid,scoring= score,cv=5,verbose=0)
grid_Rforest_2.fit(XTrain, YTrain)

In [ ]:
grid_Rforest_2.best_params_

In [ ]:
param_grid= {'max_depth': [5,10,15,20],'max_features':['auto', 'sqrt','log2'],'min_samples_leaf':[1,3,5,10]}
score='neg_mean_absolute_error'
grid_Rforest_3 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=500,random_state=123),param_grid= param_grid,scoring= score,cv=5,verbose=0)
grid_Rforest_3.fit(XTrain, YTrain)

In [ ]:
result_sc= result_sc.append(pd.DataFrame({
    'Modèle' : ['Random forest'],
    'Score_RMSE':grid_Rforest_1.best_score_,
    'Score_R2':grid_Rforest_2.best_score_,
    'Score_MAE':grid_Rforest_3.best_score_
}))

In [ ]:
result_sc

In [ ]:
from sklearn.svm import SVR #Support Vector Regression
param_grid= {'gamma' : [1e-2, 1e-3, 1e-4, 1e-5],
              'epsilon' : [0.001, 0.01, 0.1, 1], #erreur tolérée par l'algorithme
              'C' : [0.001, 0.10, 0.1, 10, 25, 50,100]} #terme de regularisation qui maximisez la marge et minimisez un terme d'erreur

svm_grid_1= GridSearchCV(estimator = SVR(),param_grid=param_grid,scoring ='neg_mean_squared_error',cv=5,verbose=0)

svm_grid_1.fit(XTrain, YTrain)

In [ ]:
svm_grid_1.best_params_

In [ ]:
svm_grid_1.best_estimator_

In [ ]:
svm_grid_1.best_score_

In [ ]:
param_grid= {'gamma' : [1e-2, 1e-3, 1e-4, 1e-5],
              'epsilon' : [0.001, 0.01, 0.1, 1],
              'C' : [0.001, 0.10, 0.1, 10, 25, 50,100]}

svm_grid_2= GridSearchCV(estimator = SVR(),param_grid=param_grid,scoring ='r2',cv=5,verbose=0)

svm_grid_2.fit(XTrain, YTrain)

In [ ]:
svm_grid_2.best_params_

In [ ]:
param_grid= {'gamma' : [1e-2, 1e-3, 1e-4, 1e-5],
              'epsilon' : [0.001, 0.01, 0.1, 1],
              'C' : [0.001, 0.10, 0.1, 10, 25, 50,100]}

svm_grid_3= GridSearchCV(estimator = SVR(),param_grid=param_grid,scoring ='neg_mean_absolute_error',cv=5,verbose=0)

svm_grid_3.fit(XTrain, YTrain)

In [ ]:
result_sc= result_sc.append(pd.DataFrame({
    'Modèle' : ['Support Vector Regressor'],
    'Score_RMSE':svm_grid_1.best_score_,
    'Score_R2':svm_grid_2.best_score_,
    'Score_MAE':svm_grid_3.best_score_

}))

In [ ]:
result_sc

In [ ]:
from xgboost import XGBRegressor
param_grid= {'n_estimators' : [10,50,100,500,1000,2000], 'learning_rate' : [1,0.1,0.01,0.001,0.0001]}
score = 'neg_mean_squared_error'
xgb_grid_1= GridSearchCV(XGBRegressor(n_jobs=-1),param_grid,scoring = score,cv = 5,verbose=0)
xgb_grid_1.fit(XTrain,YTrain)

In [ ]:
xgb_grid_1.best_params_

In [ ]:
xgb_grid_1.best_estimator_

In [ ]:
xgb_grid_1.best_score_

In [ ]:
param_grid= {'n_estimators' : [10,50,100,500,1000,2000], 'learning_rate' : [1,0.1,0.01,0.001,0.0001]}
score = 'r2'
xgb_grid_2= GridSearchCV(XGBRegressor(n_jobs=-1),param_grid,scoring = score,cv = 5,verbose=0)
xgb_grid_2.fit(XTrain,YTrain)

In [ ]:
param_grid= {'n_estimators' : [10,50,100,500,1000,2000], 'learning_rate' : [1,0.1,0.01,0.001,0.0001]}
score = 'neg_mean_absolute_error'
xgb_grid_3= GridSearchCV(XGBRegressor(n_jobs=-1),param_grid,scoring = score,cv = 5,verbose=0)
xgb_grid_3.fit(XTrain,YTrain)

In [ ]:
result_sc= result_sc.append(pd.DataFrame({
    'Modèle' : ['XGBoost'],
    'Score_RMSE':xgb_grid_1.best_score_,
    'Score_R2':xgb_grid_2.best_score_,
    'Score_MAE':xgb_grid_3.best_score_
}))

Comparons les modèles sur les critères de la RMSE, R2 et MAE


In [ ]:
result_sc

In [ ]:
result_sc[['Score_RMSE','Score_MAE']]= abs(result_sc[['Score_RMSE','Score_MAE']])

In [ ]:
#result_sc.set_index(['Modèle'],inplace=True)
result_sc.T.plot(kind='bar',title='Comparaison des performances des modèles',figsize=(8,8))

Le random forest et le XGBoost sont les modeles avec les plus faibles RMSE et le plus haut R2 avec des valeurs tres proches

Temps de prediction pour chaque model :

In [ ]:
%timeit svm_grid_1.predict(XTest)

In [ ]:
%timeit xgb_grid_1.predict(XTest)

In [ ]:
%timeit grid_Rforest_1.predict(XTest)

In [ ]:
%timeit grid_elastic_1.predict(XTest)

le modele qui predire le plus rappidement est XGBoost

le modele finale selectionnée pour predire l'emission est le XGBoost.

2- Entrainement du modele finale selectionnée Avec Energystarscore

In [ ]:
df2['ENERGYSTARScore'] = df2['ENERGYSTARScore'].astype('float')

In [ ]:
X2_es= df2.drop(['log_GHGEmissionsIntensity'],axis=1)
Y2_es= df2['log_GHGEmissionsIntensity']
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X2_es, Y2_es, test_size=0.3)
# standardiser les données
std_scale = StandardScaler()
X_Train = std_scale.fit_transform(X_Train) #X_train_std
X_Test = std_scale.transform(X_Test)#X_test_std

In [ ]:
param_grid= {'n_estimators' : [10,50,100,500,1000,2000,3000], 'learning_rate' : [1,0.1,0.01,0.001,0.0001]}
score = 'neg_mean_squared_error'
xgb_grid_E= GridSearchCV(XGBRegressor(n_jobs=-1),param_grid,scoring = score,cv = 5,verbose=0)
xgb_grid_E.fit(X_Train,Y_Train)

In [ ]:
xgb_grid_E.best_params_

In [ ]:
xgb_grid_E.best_estimator_

In [ ]:
xgb_grid_E.best_score_

In [ ]:
%timeit xgb_grid_E.predict(X_Test)

In [ ]:
df_xgb_E= pd.DataFrame({'Features' : X2_es.columns,'Features_importances' : xgb_grid_E.best_estimator_.feature_importances_})

In [ ]:
df_xgb_E.sort_values(by='Features_importances', ascending=False,inplace=True)
plt.figure(figsize=(10,10))
plt.title('XGB - Importance des Features')
sns.barplot(y = df_xgb_E['Features'],x = (df_xgb_E['Features_importances']))
plt.show()

In [ ]:
prediction_y= xgb_grid_E.best_estimator_.predict(X_Test)
pd.DataFrame({'emissions' : Y_Test, 'prediction_emission' : prediction_y})

In [ ]:
plt.figure(figsize=(8,8))
plt.title('Comparaison des emissions prédictes et réels')
sns.lineplot([-6,6],[-6,6], color='r')
sns.scatterplot(Y_Test,prediction_y)
plt.show()

**Conclusion**

- Le modèle final retenu pour la prediction de consommation est le XGBoost entraîné sur toutes les features et avec les paramètres optimal suivants: {'learning_rate': 0.01, 'n_estimators': 2000}  
- Le modèle finale retenu pour la prediction d'émissions est  XGBoost entraîné sur toutes les features(avec energy star score) , et avec les paramètres optimal suivants: {'learning_rate': 0.01, 'n_estimators': 3000}
- L'utilisation de l'energy star score à ameliore la performance du modele.
